In [ ]:
###############################################################################
import aliases

In [ ]:
import functools as _functools

from everest.incision import IncisionProtocol as _IncisionProtocol
from everest.utilities import caching as _caching

from everest.ptolemaic.chora import (
    Chora as _Chora,
    Slyce as _Slyce,
    ChainChora as _ChainChora,
    Degenerate as _Degenerate,
    )
from everest.ptolemaic.fundaments.floatt import Floatt as _Floatt
from everest.ptolemaic.fundaments.brace import Brace as _Brace
from everest.ptolemaic.schema import Schema as _Schema
from everest.ptolemaic.sprite import Sprite as _Sprite
from everest.ptolemaic.essence import Essence as _Essence
from everest.ptolemaic.eidos import Eidos as _Eidos
from everest.ptolemaic.diict import Kwargs as _Kwargs

In [ ]:
class Traversable(_Chora, metaclass=_Schema):


    MROCLASSES = ('Slyce', 'Instruments', 'Case', 'Config', 'Metric')

    @classmethod
    def __class_init__(cls, /):

        super().__class_init__()

        for name in cls.Instruments.SUBCLASSES:

            @_functools.wraps(getattr(cls.Instruments, name).__call__)
            def method(self, /, *args, _name_=name, **kwargs):
                return getattr(self.Instruments, _name_)(
                    self, *args, **kwargs
                    )

            setattr(cls, name.lower(), method)

        def slyce(self, /, *args, **kwargs):
            return self.Slyce(self, *args, **kwargs)

        cls.slyce = slyce

    @property
    def __incision_manager__(self, /):
        return self.slyce(self.Case, self.Config, self.Metric)


    class Slyce(_ChainChora, metaclass=_Sprite):

        schematic: object
        case: 'owner.Case.Oid'
        config: 'owner.Config.Oid'
        metric: 'owner.Metric.Oid'

        @property
        @_caching.soft_cache()
        def __incision_manager__(self, /):
            return _Brace[_Kwargs(
                case=self.case, config=self.config, metric=self.metric
                )]

        def __incise_slyce__(self, incisor: _Brace.Oid, /):
            cs, cg, mc = incisor.choras
            schematic = self.schematic
            if incisor.active == (False, False, True):
                return schematic.traverse(cs.retrieve(), cg.retrieve(), mc)
            out = schematic.slyce(*incisor.choras)
            out.softcache['__incision_manager__'] = incisor
            return out

        def __incise_retrieve__(self, incisor: _Brace, /):
            return self.schematic.stage(*incisor)


    class Instruments(metaclass=_Sprite):


        SUBCLASSES = ('Line', 'Traverse', 'Stage')

        schematic: 'owner'


        class Line(_ChainChora, metaclass=_Essence):

            case: 'owner.Case'
            initial: 'owner.Config'

            def traverse(self, /, interval):
                return self.schematic.traverse(self, interval)

            def stage(self, /, metric):
                return self.schematic.stage(self, metric)

            @property
            def __incision_manager__(self, /):
                return self.schematic.Metric

            @property
            def __incise_slyce__(self, /):
                return self.traverse

            @property
            def __incise_retrieve__(self, /):
                return self.stage


        class Traverse(metaclass=_Essence):

            case: 'owner.Case'
            initial: 'owner.Config'
            interval: 'owner.Metric.Oid'

            @property
            def __incision_manager__(self, /):
                return self.interval

            @property
            def __incise_slyce__(self, /):
                return self.line.traverse

            @property
            def __incise_retrieve__(self, /):
                return self.line.stage


        class Stage(metaclass=_Essence):

            case: 'owner.Case'
            initial: 'owner.Config'
            metric: 'owner.Metric'


    class Case(metaclass=_Eidos):

        a: _Floatt
        b: _Floatt
        c: _Floatt


    class Config(metaclass=_Eidos):

        x: _Floatt
        y: _Floatt
        z: _Floatt


    class Metric(metaclass=_Eidos):

        chron: _Floatt

In [ ]:
model = Traversable()

In [ ]:
model[0., 1.]

In [ ]:
model[(0., 1., 2.), (10., 20., 30.), 4.]

In [ ]:
model[(0., 1., 2.), (10., 20., 30.), 4.:]

In [ ]:
#     class Slyce(_Slyce):

#         MULTICHANNELS = (
#             'brace',
#             'braceoid',
#             )

#         def handle_brace(self, incisor: _Brace, /, *, caller):
#             return self.bracegetmeths[tuple(map(type, incisor))](
#                 self, incisor, caller=caller
#                 )

#         def handle_braceoid(self,
#                 incisor: _Brace.Oid.__mroclass_basis__, /, *, caller
#                 ):
#             return self.braceoidgetmeths[tuple(map(type, incisor.choras))](
#                 self, incisor, caller=caller
#                 )

#         def brace_retrieve_simple(self,
#                 incisor: (object, object, object), /
#                 ):
#             cs, cg, mc = incisor
#             return (
#                 self.bound.Line(cs, cg)
#                 .__incise_retrieve__(mc)
#                 )

#         def braceoid_slyce_(self,
#                 incisor: (object, object, object), /
#                 ):
#             return self._ptolemaic_class__(self.bound, incisor)

#         def braceoid_catch_all(self,
#                 incisor: (object, object, object), /, *, caller
#                 ):
#             return _IncisionProtocol.SLYCE(caller)(
#                 self._ptolemaic_class__(self.bound, incisor)
#                 )